# Nhanes Inferential HW
## Imported the data for the follwing Categories

- Marital Status (DMDMARTZ)
- Education Level (DMDEDUC2)
- Age in Years (RIDAGEYR)
- Systolic Blood Pressure (BPXOSY3)
- Diastolic Blood Pressure (BPXODI3)
- Vitamin D Lab Interpretation (LBDVD2LC)
- Hepatitis B Lab Antibodies (LBXHBS)
- Weak/Failing Kidneys (KIQ022)
- Minutes of Sedentary Behavior (PAD680)
- Current Self-Reported Weight (WHD020)

In [140]:
import pandas as pd

from scipy.stats import ttest_rel
demo = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/DEMO_L.XPT', format='xport') #, encoding='latin1')  # Try 'latin1' or 'iso-8859-1' if UTF-8 fails
exam = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/BPXO_L.XPT', format='xport')
labd = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/VID_L.XPT', format='xport')
lab_hep = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/HEPB_S_L.XPT', format='xport')
sedentary = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/PAQ_L.XPT', format='xport')
sr_weight = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/WHQ_L.XPT', format='xport')
kidney = pd.read_sas('https://wwwn.cdc.gov/Nchs/Nhanes/2021-2022/KIQ_U_L.XPT', format='xport')


# Question 1
## "Is there an association between marital status (married or not married) and education level (bachelor’s degree or higher vs. less than a bachelor’s degree)?"

## Steps
1. Imported Chi2 package
2. Cleaned up married data. Split into 2 categories "Married" and "Not Married"
3. Cleaned up education data. Split into 2 categories "less than a bachelors" and "Bachelors"
4. Used CHI-SQUARED test to see if there was an association between Marital status and education level
5. Results showed that there was a significant association between marital status and education level. based on my p-value being < .05

In [143]:
# Question 1 "Is there an association between marital status (married or not married) and education level (bachelor’s degree or higher vs. less than a bachelor’s degree)?"
from scipy.stats import chi2_contingency

clean_demo_mar = demo.loc[demo['DMDMARTZ'] < 4]
clean_demo_mar2 = clean_demo_mar['DMDMARTZ'].replace({1: 'Married', 2: 'Not Married', 3: 'Not Married'})

print(clean_demo_mar2.value_counts())
clean_demo_edu = demo.loc[demo['DMDEDUC2'] < 6]
clean_demo_edu2 = clean_demo_edu['DMDEDUC2'].replace({1: 'Less than a Bachelor’s', 2: 'Less than a Bachelor’s', 3: 'Less than a Bachelor’s', 4: 'Less than a Bachelor’s', 5: 'Bachelor’s'})

contingency_table = pd.crosstab(clean_demo_mar2, clean_demo_edu2)
contingency_table
#demo['DMDEDUC2']

stat, p_value, dof, expected = chi2_contingency(contingency_table)
print(f"Chi-squared statistic: {stat}")
print(f"P-value: {p_value}")
print("Expected frequencies:")
print(expected)

if p_value > 0.05:
    print("There is no significant association between marital status and education level.")
else:
    print("There is a significant association between marital status and education level.")

DMDMARTZ
Married        4136
Not Married    3647
Name: count, dtype: int64
Chi-squared statistic: 129.17381826322236
P-value: 6.213197986632679e-30
Expected frequencies:
[[1394.13458569 2739.86541431]
 [1226.86541431 2411.13458569]]
There is a significant association between marital status and education level.


# Question 2
## "Is there a difference in the mean sedentary behavior time between those who are married and those who are not married?"

## Steps
1. Imported ttest package
2. Cleaned up married data. Split into 2 categories 1.0 = Married, 2.0 = Not married  
3. Cleaned up sedentary behavior removed values 7777, 9999, and null.
4. Ran a ttest on data
5. Results gave that there was Significant difference between sedentary behavior time between those who are married and those who are not married.

In [131]:
# Question 2: "Is there a difference in the mean sedentary behavior time between those who are married and those who are not married?"
from scipy.stats import ttest_rel

clean_demo_mar = demo.loc[demo['DMDMARTZ'] < 4]
clean_demo_mar3 = clean_demo_mar['DMDMARTZ'].replace({1: 1.0, 2: 2.0, 3: 2.0})
clean_sedentary = sedentary.loc[sedentary['PAD680'] <= 1380]
clean_sedentary2 = clean_sedentary['PAD680']

stat, p_value = ttest_rel(clean_demo_mar3,clean_sedentary2.head(7783))
print(f"Paired t-Test Statistic: {stat}, p-value: {p_value}")

# Interpretation
if p_value > 0.05:
    print("No significant difference between sedentary behavior time between those who are married and those who are not married.")
else:
    print("Significant difference between sedentary behavior time between those who are married and those who are not married.")

Paired t-Test Statistic: -151.21992508249474, p-value: 0.0
Significant difference between sedentary behavior time between those who are married and those who are not married.


# Question 3

## How do age and marital status affect systolic blood pressure?

## Steps
1. Import neccessary packages
2. Create variables for age and blood pressure (Already have data for marital status)
3. Run a 2 way anova to see how age and marital status affect systolic blood pressure.
4. Based on the results p-value is < .05 therefor we can conclude age and marital status have affect on systolic blood pressure

In [137]:
# Question 3: "How do age and marital status affect systolic blood pressure?"
import statsmodels.api as sm
from statsmodels.formula.api import ols

Demo_age = demo['RIDAGEYR']
clean_demo_mar3

exam_sys = exam['BPXOSY3']
exam_sys2 = exam_sys.dropna()

anova_table = pd.concat([Demo_age, clean_demo_mar3, exam_sys2], axis=1)
anova_table

model = ols('BPXOSY3 ~ RIDAGEYR + DMDMARTZ', data=anova_table).fit()
table = sm.stats.anova_lm(model, typ=2)
print(table)
print(model.summary())

if p_value > 0.05:
    print("No significant affectivness between age and marital status on systolic blood pressure.")
else:
    print("Significant affectiveness between age and marital status on systolic blood pressure.")


                sum_sq      df         F    PR(>F)
RIDAGEYR  1.073427e+02     1.0  0.306684  0.579748
DMDMARTZ  3.193353e+00     1.0  0.009124  0.923908
Residual  1.705604e+06  4873.0       NaN       NaN
                            OLS Regression Results                            
Dep. Variable:                BPXOSY3   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.1561
Date:                Tue, 05 Nov 2024   Prob (F-statistic):              0.856
Time:                        16:51:22   Log-Likelihood:                -21199.
No. Observations:                4876   AIC:                         4.240e+04
Df Residuals:                    4873   BIC:                         4.242e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
      

# Question 4
## Is there a correlation between self-reported weight and minutes of sedentary behavior?

## Steps
1. Import packages for both correlation tests
2. Clean data weight data does not contain values 7777, 9999, and null
3. Use clean sedentary data created from above
4. Run Person correlation test
5. Based on results did not see any significant correlation between self-reported weight and minutes of sedentary behavior.
6. Try to run spearmen correlation
7. Still got No significant correlation between self-reported weight and minutes of sedentary behavior.

## Note: I used the head function to make my arrays equal before the test was run. Will this significantly affect the results of the tests I run?




In [138]:
# Question 4: "Is there a correlation between self-reported weight and minutes of sedentary behavior?"

from scipy.stats import pearsonr
from scipy.stats import spearmanr
clean_sr_weight = sr_weight.loc[sr_weight['WHD020'] <= 530]
clean_sr_weight2 = clean_sr_weight['WHD020']

clean_sedentary2 = clean_sedentary['PAD680']

# Calculate Pearson correlation
correlation, p_value = pearsonr(clean_sedentary2, clean_sr_weight2.head(8065))
print(f"Correlation coefficient: {correlation}, p-value: {p_value}")

# Interpretation
if p_value > 0.05:
    print("No significant correlation between self-reported weight and minutes of sedentary behavior.")
else:
    print("Significant correlation between self-reported weight and minutes of sedentary behavior.")


# Calculate Spearman correlation
corr, p_value = spearmanr(clean_sedentary2, clean_sr_weight2.head(8065))
print(f"Spearman Correlation Coefficient: {corr}, p-value: {p_value}")

# Interpretation
if p_value > 0.05:
    print("No significant monotonic relationship.")
else:
    print("Significant monotonic relationship.")

Correlation coefficient: -0.000772437025182364, p-value: 0.944704475162349
No significant correlation between self-reported weight and minutes of sedentary behavior.
Spearman Correlation Coefficient: -3.17674278940831e-05, p-value: 0.9977240831576109
No significant monotonic relationship.


# Question 5
## Is there a correlation between Systolic blood pressure and weak/failing kidneys?

### Reason for this test generally uncontrolled high systolic blood pressure can lead to kidney disease.

## Steps
1. import t-test package
2. clean kidney data into 2 categories 1.0 = Yes, 2.0 = No
3. Use previously cleaned systolic blood pressure
4. Run t-test
5. based on t-test I got that there was a Significant correlation between Systolic blood pressure and weak/failing kidneys.

## Note: I used the head function to make my arrays equal before the test was run. Will this significantly affect the results of the tests I run?

In [142]:
# Question 5: "Is there a correlation between Systolic blood pressure and weak/failing kidneys?"
from scipy.stats import ttest_rel
clean_kidney = kidney.loc[kidney['KIQ022'] < 3]
clean_kidney = clean_kidney['KIQ022']
#print(clean_kidney.count())
#print(exam_sys2.count())


stat, p_value = ttest_rel(exam_sys2, clean_kidney.head(7480))
print(f"Correlation coefficient: {correlation}, p-value: {p_value}")

# Interpretation
if p_value > 0.05:
    print("No significant correlation between Systolic blood pressure and weak/failing kidneys.")
else:
    print("Significant correlation between Systolic blood pressure and weak/failing kidneys.")


7794
7480
Correlation coefficient: -0.000772437025182364, p-value: 0.0
Significant correlation between Systolic blood pressure and weak/failing kidneys.
